In [ ]:
# Saurabh Chatterjee  22EE65R14

In [40]:
import IPython.display as display
import librosa
import librosa.display
import numpy as np
from seglearn.transform import Segment
from scipy.fft import dct , idct
import numpy as np
from sklearn.decomposition import PCA
import math

In [41]:
# Loading Audio File

sample_audio = 'C:\\Users\\Saurabh Chatterjee\\the_godfather_theme_instru_mix.wav'
x, sr = librosa.load(sample_audio , sr =44100)
x.shape

(2648184,)

In [42]:
# Slicing the sample audio

x_v = x[: sr *30]
x_g = x[sr *30: sr *60]
X = np.stack (( x_v ,x_g))
X.shape

(2, 1323000)

In [43]:
y = np.array([0 ,1])   # Labels for two classes
y.shape

(2,)

In [44]:
samples_per_frame = int(sr *0.02)           # number of samples in a single frame of 0.02 seconds
samples_per_frame

882

In [47]:
# Segment audio signal

ts = Segment(width = samples_per_frame, overlap =0).fit(X,y)
s_d = ts.transform(X,y)
np.shape(s_d)

(3,)

In [48]:
s_d

(array([[ 0.04036909,  0.04036748,  0.04083532, ...,  0.0251925 ,
          0.02481925,  0.02450895],
        [ 0.02474099,  0.02545887,  0.02478904, ...,  0.03188771,
          0.03228593,  0.03355002],
        [ 0.03435582,  0.03636414,  0.03981543, ...,  0.05857033,
          0.0597496 ,  0.06024611],
        ...,
        [-0.17038906, -0.16573942, -0.16168225, ..., -0.19248325,
         -0.19756675, -0.20212317],
        [-0.20609367, -0.20906985, -0.21107638, ...,  0.03966218,
          0.03349531,  0.02745306],
        [ 0.02146798,  0.0153572 ,  0.00915354, ...,  0.05985272,
          0.05267054,  0.04531533]]),
 array([0, 0, 0, ..., 1, 1, 1]),
 None)

In [49]:
X_d = dct(s_d[0])   # DFT along 2nd axis (samples axis)
X_d.shape

(3000, 882)

In [50]:
X_d

array([[-2.48189759e+00, -4.81813007e+00, -1.20087913e+01, ...,
        -6.76845352e-05, -2.51825449e-05, -1.18895649e-04],
       [ 9.56470490e-01, -3.08480941e+00,  1.05587758e+00, ...,
         2.24445520e-05,  2.73213618e-06, -8.82152744e-05],
       [-1.62404287e+00, -2.50365709e-01,  1.25386461e+00, ...,
         8.88671176e-05,  5.58137802e-05, -4.35264310e-05],
       ...,
       [ 1.70405452e+01,  1.75428174e+00,  2.01961125e+01, ...,
         4.10872381e-05, -5.49206522e-05, -3.54118227e-05],
       [-6.22604358e+00, -8.77214528e+00, -9.16791565e+00, ...,
         1.44522299e-05,  9.76971659e-09, -9.07817074e-06],
       [ 2.27702594e+00, -6.51089741e+00,  1.74554074e+00, ...,
         1.67227057e-05, -1.59470201e-05, -3.39529357e-05]])

In [51]:
x_train , y_train = X_d , s_d [1]
from sklearn.utils import shuffle
x_train , y_train = shuffle ( x_train , y_train , random_state =0)

In [52]:
x_train.shape

(3000, 882)

In [128]:
y_train.shape

(3000,)

In [54]:
x_train_0 = x_train[y_train==0]
x_train_1 = x_train[y_train==1]


In [55]:
x_train_0.shape

(1500, 882)

In [134]:
#Prior Probabilities of Class 0 & 1

prior_0 = y_train.tolist().count(0) / len(y_train)
print("Prior of Class 0: ", prior_0)

prior_1 = y_train.tolist().count(1) / len(y_train)
print("Prior of Class 1: ", prior_1)

Prior of Class 0:  0.5
Prior of Class 1:  0.5


In [223]:
# Class 0

mean_0 = np.mean(x_train_0, axis=0)
print("Mean Vector Class 0 Shape: ", mean_0.shape)

x_train_0_zero_mean = x_train_0 - np.mean(x_train_0, axis=0, keepdims=True)

R_0 = 1/1500 * np.matmul(x_train_0_zero_mean.T, x_train_0_zero_mean)
print("Original Covariance Matrix Shape of Class 0: ", R_0.shape)

det_R_0 = np.linalg.det(R_0)
det_R_0

Mean Vector Class 0 Shape:  (882,)
Original Covariance Matrix Shape of Class 0:  (882, 882)


0.0

In [57]:
# Zero Determinant Coming

In [58]:
np.linalg.matrix_rank(R_0)   # Rank is one less than 882 - so determinant zero - Use PCA to reduce dimension

881

In [225]:
# Class 1

mean_1 = np.mean(x_train_1, axis=0)
print("Mean Vector Shape: ", mean_1.shape)

x_train_1_zero_mean = x_train_1 - np.mean(x_train_1, axis=0, keepdims=True)

R_1 = 1/1500 * np.matmul(x_train_1_zero_mean.T, x_train_1_zero_mean)
print("Original Covariance Matrix Shape of Class 0: ", R_1.shape)

det_R_1 = np.linalg.det(R_1)
det_R_1

Mean Vector Shape:  (882,)
Original Covariance Matrix Shape of Class 0:  (882, 882)


0.0

In [60]:
np.linalg.matrix_rank(R_1)   # Rank is less than 882 - so determinant zero - Use PCA to reduce dimension

750

In [204]:
# Using PCA to Reduce Dimensions of both "x_train_0" & "x_train_1"

# Let's Reduce the Dimension to 70 features so that both the Covariance Determinants don't become zero
# Taking 70 features because for more number of  features the determinant is becoming zero because of some same eigen values
# and so some dependent eigen vectors

d = 70
pca = PCA(n_components = 70)

In [205]:
# Class 0

  
x_train_0_t = pca.fit_transform(x_train_0)
print(x_train_0_t.shape)

R_0_t = 1/1500 * np.matmul(x_train_0_t.T, x_train_0_t)
R_0_t.shape

det_R_0_t = np.linalg.det(R_0_t)
print(det_R_0_t)

R_0_t_inv = np.linalg.inv(R_0_t)
R_0_t_inv

(1500, 70)
4.794766316719772e+139


array([[ 3.93769954e-04, -1.29681317e-21,  5.99678727e-20, ...,
         1.28518697e-19, -1.95150661e-19,  1.71199485e-19],
       [-1.29681317e-21,  4.17813600e-04, -2.79515387e-20, ...,
         9.97800484e-21,  2.01101705e-19,  1.90645668e-19],
       [ 5.99678727e-20, -2.79515387e-20,  8.97034451e-04, ...,
         7.42646853e-19,  2.85400611e-19,  3.82281095e-19],
       ...,
       [ 1.28518697e-19,  9.97800484e-21,  7.42646853e-19, ...,
         5.25159178e-02, -1.28526708e-17,  8.19478333e-19],
       [-1.95150661e-19,  2.01101705e-19,  2.85400611e-19, ...,
        -1.28526708e-17,  5.38185814e-02, -1.17572783e-17],
       [ 1.71199485e-19,  1.90645668e-19,  3.82281095e-19, ...,
         8.19478333e-19, -1.17572783e-17,  5.67962388e-02]])

In [206]:
# Class 1

  
x_train_1_t = pca.fit_transform(x_train_1)
print(x_train_1_t.shape)

R_1_t = 1/1500 * np.matmul(x_train_1_t.T, x_train_1_t)
R_1_t.shape

det_R_1_t = np.linalg.det(R_1_t)
print(det_R_1_t)

R_1_t_inv = np.linalg.inv(R_1_t)
R_1_t_inv

(1500, 70)
2.2885337848729557e+106


array([[ 1.29884714e-04, -7.14177383e-20, -8.15170865e-21, ...,
         5.15141494e-19,  2.07566000e-19,  1.11312329e-18],
       [-7.14177383e-20,  1.34140623e-04, -5.46706573e-20, ...,
        -8.66717071e-19, -3.45256080e-19,  1.85872474e-20],
       [-8.15170865e-21, -5.46706573e-20,  1.74007265e-04, ...,
         2.50721000e-19,  2.00171524e-19,  6.41212122e-19],
       ...,
       [ 5.15141494e-19, -8.66717071e-19,  2.50721000e-19, ...,
         6.18459160e-01,  1.81800468e-16, -2.92566218e-16],
       [ 2.07566000e-19, -3.45256080e-19,  2.00171524e-19, ...,
         1.81800468e-16,  7.13998691e-01,  1.04005340e-16],
       [ 1.11312329e-18,  1.85872474e-20,  6.41212122e-19, ...,
        -2.92566218e-16,  1.04005340e-16,  7.69789919e-01]])

In [207]:
# Test Data:


test_audio = 'C:\\Users\\Saurabh Chatterjee\\the_godfather_theme_test.wav'
x_test_data, sr = librosa.load(test_audio , sr =44100)
print("Length of Test Audio Data: ", x_test_data.shape)


x_test_v = x_test_data[: sr *15]
x_test_g = x_test_data[sr *15: sr *30]
X_test = np.stack(( x_test_v ,x_test_g))
print("Test Data Divided into Half for each Class, Shape: ",X_test.shape)


Y_test = np.array([1 ,0])

x_test_ts = Segment(width = samples_per_frame, overlap =0)
X_test_seg = x_test_ts.fit_transform(X_test, Y_test)         # Tuple of X_test and Label y
print("Test Data Segmented: ", X_test_seg)

print("Segmented Test Data Shape: ", X_test_seg[0].shape)
print("Segmented Test Data Label Shape: ", X_test_seg[1].shape)

X_test_d = dct(X_test_seg[0])   # DFT along 2nd axis (samples axis)

x_test , y_test  = X_test_d, X_test_seg[1]

print("Test Data Shape: ", x_test.shape)
print("Test Data Label: ", y_test.shape)

Length of Test Audio Data:  (1323006,)
Test Data Divided into Half for each Class, Shape:  (2, 661500)
Test Data Segmented:  (array([[-5.96046448e-08,  5.96046448e-08,  5.96046448e-08, ...,
         1.81024671e-02,  1.43143535e-02,  1.03410482e-02],
       [ 6.29204512e-03,  2.11083889e-03, -2.26145983e-03, ...,
        -9.87550616e-02, -9.77470875e-02, -9.61292386e-02],
       [-9.35562849e-02, -9.01816487e-02, -8.64366293e-02, ...,
         1.88408494e-02,  2.14352608e-02,  2.39666700e-02],
       ...,
       [-2.48120368e-01, -2.43453264e-01, -2.39868879e-01, ...,
        -2.97048807e-01, -2.95518398e-01, -2.96383083e-01],
       [-2.98770010e-01, -3.01415741e-01, -3.02167773e-01, ...,
        -2.80164003e-01, -2.85213411e-01, -2.90459275e-01],
       [-2.93141425e-01, -2.93460131e-01, -2.91887164e-01, ...,
        -1.38161659e-01, -1.45518959e-01, -1.52980745e-01]]), array([1, 1, 1, ..., 0, 0, 0]), None)
Segmented Test Data Shape:  (1500, 882)
Segmented Test Data Label Shape:  (150

In [208]:
# PCA on Test Data to Compress to d=400 
x_test_t = pca.fit_transform(x_test)
x_test_t.shape

(1500, 70)

In [209]:
x_test_t.shape[0]

1500

In [210]:
# Posterior Probability:


# Class 0:

p_xtest_0 = []

for i in range(x_test_t.shape[0]):
    x_test_i = x_test_t[i,:]
    #p_xtest_0[i] = math.log(1/((2*math.pi)**(d/2) * det_R_0_t**(1/2))) + (-1/2)*(x_test_i @ R_0_t_inv @ x_test_i.T)
    likelihood_0 = 1/((2*math.pi)**(d/2) * det_R_0_t**(1/2)) * np.exp((-1/2)*(x_test_i @ R_0_t_inv @ x_test_i.T))
    
    likelihood_1 = 1/((2*math.pi)**(d/2) * det_R_1_t**(1/2)) * np.exp((-1/2)*(x_test_i @ R_1_t_inv @ x_test_i.T))
    
    posterior_0 = likelihood_0 * prior_0 / (likelihood_0 * prior_0 + likelihood_1 * prior_1)
    p_xtest_0.append(posterior_0)
    


len(p_xtest_0)

1500

In [211]:
y_test_predict = []

for i in range(len(p_xtest_0)):
    if p_xtest_0[i] < 0.5:
        y_test_predict.append(1)
    else:
        y_test_predict.append(0)

y_test_predict

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [216]:
# Classifier Error:

error = abs(y_test_predict - y_test)

Error_Count = error.tolist().count(1)
Error_Percentage = Error_Count / len(error) * 100

Error_Percentage

12.266666666666666

In [ ]:
# Saurabh Chatterjee  22EE65R14